In [3]:
!git clone https://github.com/Magistri-Codicis/dtw-capcha.git
%cd dtw-capcha
%pip install -r requirements.txt

C:\Users\Hund\Desktop\Jupyter\dtw-capcha


fatal: destination path 'dtw-capcha' already exists and is not an empty directory.
C:\Python311\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow<=2.10 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.15.0rc0, 2.15.0rc1)
ERROR: No matching distribution found for tensorflow<=2.10

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from collections import Counter

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.utils.generic_utils import register_keras_serializable, get_custom_objects

from configs import ModelConfigs

print("Tensorflow version: ", tf.__version__)

seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)
configs = ModelConfigs()

ModuleNotFoundError: No module named 'pandas'

In [4]:
# Path to the data directory
data_dir = Path(configs.data_dir)

# Get list of all the images
images = []
for line in open(os.path.join(data_dir, "index.txt")):
    parts = line.split(" ")
    images.append(os.path.join(data_dir, str.strip(parts[1]).replace('\\', '/')))
print("Number of images found: ", len(images))

# Let's take a look at some samples first. 
# Always look at your data!
sample_images = images[:4]

_, ax = plt.subplots(2, 2, figsize=(configs.width, configs.height))
for i in range(configs.samples):
    print("Image Path: ", str(sample_images[i]), os.path.exists(str(sample_images[i])))
    img = cv2.imread(str(sample_images[i]))
    print("Shape of image: ", img.shape)
    ax[i // 2, i % 2].imshow(img)
    ax[i // 2, i % 2].axis('off')
plt.show()

Number of images found:  5000
Image Path:  samples\captcha-4\1.png True
Shape of image:  (40, 400, 3)
Image Path:  samples\captcha-4\2.png True
Shape of image:  (40, 400, 3)
Image Path:  samples\captcha-4\3.png True
Shape of image:  (40, 400, 3)
Image Path:  samples\captcha-4\4.png True
Shape of image:  (40, 400, 3)


In [5]:
# Store all the characters in a set
characters = set()

# A list to store the length of each captcha
captcha_length = []

# Store image-label info
dataset = []

# Iterate over the dataset and store the
# information needed
for line in open(os.path.join(data_dir, "index.txt")):
    parts = line.split(" ")
    # 1. Get the label associated with each image
    img_path = os.path.join(data_dir, str.strip(parts[1]).replace('\\', '/'))
    label = str.strip(parts[0])
    # 2. Store the length of this cpatcha
    captcha_length.append(len(label))
    # 3. Store the image-label pair info
    dataset.append((str(img_path), label))

    # 4. Store the characters present
    for ch in label:
        characters.add(ch)

# Sort the characters        
characters = sorted(characters)

# Convert the dataset info into a dataframe
dataset = pd.DataFrame(dataset, columns=["img_path", "label"], index=None)

# Shuffle the dataset
dataset = dataset.sample(frac=1.).reset_index(drop=True)

print("Number of unqiue charcaters in the whole dataset: ", len(characters))
print("Maximum length of any captcha: ", max(Counter(captcha_length).keys()))
print("Characters present: ", characters)
print("Total number of samples in the dataset: ", len(dataset))
dataset.head()

Number of unqiue charcaters in the whole dataset:  12
Maximum length of any captcha:  15
Characters present:  ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
Total number of samples in the dataset:  5000


,img_path,label
0,samples\captcha-4\2707.png,DAFGHFIKDBDBGHK
1,samples\captcha-4\2437.png,JKDAJHFHDGLICHL
2,samples\captcha-4\1202.png,FIKGCIDLEHJEJAG
3,samples\captcha-4\1487.png,ABCEEBBFFIDKACL
4,samples\captcha-4\4287.png,KIJCFLDGCIIBDIC


In [6]:
# Split the dataset into training and validation sets
training_data, validation_data = train_test_split(dataset, test_size=0.1, random_state=seed)

training_data = training_data.reset_index(drop=True)
validation_data = validation_data.reset_index(drop=True)

print("Number of training samples: ", len(training_data))
print("Number of validation samples: ", len(validation_data))

# Map text to numeric labels 
char_to_labels = {char: idx for idx, char in enumerate(characters)}

# Map numeric labels to text
labels_to_char = {val: key for key, val in char_to_labels.items()}


# Sanity check for corrupted images
def is_valid_captcha(captcha):
    for ch in captcha:
        if not ch in characters:
            return False
    return True


# Store arrays in memory as it's not a muvh big dataset
def generate_arrays(df, resize=True, img_height=configs.height, img_width=configs.width):
    """Generates image array and labels array from a dataframe.
    
    Args:
        df: dataframe from which we want to read the data
        resize (bool)    : whether to resize images or not
        img_weidth (int): width of the resized images
        img_height (int): height of the resized images
        
    Returns:
        images (ndarray): grayscale images
        labels (ndarray): corresponding encoded labels
    """

    num_items = len(df)
    images = np.zeros((num_items, img_height, img_width), dtype=np.float32)
    labels = [0] * num_items

    for i in range(num_items):
        img = cv2.imread(df["img_path"][i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        if resize:
            img = cv2.resize(img, (img_width, img_height))

        img = (img / 255.).astype(np.float32)
        label = df["label"][i]

        # Add only if it is a valid captcha
        if is_valid_captcha(label):
            images[i, :, :] = img
            labels[i] = label

    return images, np.array(labels)


# Build training data
training_data, training_labels = generate_arrays(df=training_data)
print("Number of training images: ", training_data.shape)
print("Number of training labels: ", training_labels.shape)

# Build validation data
validation_data, validation_labels = generate_arrays(df=validation_data)
print("Number of validation images: ", validation_data.shape)
print("Number of validation labels: ", validation_labels.shape)

Number of training samples:  4500
Number of validation samples:  500
Number of training images:  (4500, 40, 400)
Number of training labels:  (4500,)
Number of validation images:  (500, 40, 400)
Number of validation labels:  (500,)


In [7]:
class DataGenerator(keras.utils.Sequence):
    """Generates batches from a given dataset.
    
    Args:
        data: training or validation data
        labels: corresponding labels
        char_map: dictionary mapping char to labels
        batch_size: size of a single batch
        img_width: width of the resized
        img_height: height of the resized
        downsample_factor: by what factor did the CNN downsample the images
        max_length: maximum length of any captcha
        shuffle: whether to shuffle data or not after each epoch
    Returns:
        batch_inputs: a dictionary containing batch inputs 
        batch_labels: a batch of corresponding labels 
    """

    def __init__(self,
                 data,
                 labels,
                 char_map,
                 batch_size=16,
                 img_width=configs.width,
                 img_height=configs.height,
                 downsample_factor=configs.downsample_factor,
                 max_length=configs.max_text_length,
                 shuffle=configs.shuffle
                 ):
        self.data = data
        self.labels = labels
        self.char_map = char_map
        self.batch_size = batch_size
        self.img_width = img_width
        self.img_height = img_height
        self.downsample_factor = downsample_factor
        self.max_length = max_length
        self.shuffle = shuffle
        self.indices = np.arange(len(data))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, idx):
        # 1. Get the next batch indices
        curr_batch_idx = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]

        # 2. This isn't necessary but it can help us save some memory
        # as not all batches the last batch may not have elements
        # equal to the batch_size 
        batch_len = len(curr_batch_idx)

        # 3. Instantiate batch arrays
        batch_images = np.ones((batch_len, self.img_width, self.img_height, 1),
                               dtype=np.float32)
        batch_labels = np.ones((batch_len, self.max_length), dtype=np.float32)
        input_length = np.ones((batch_len, 1), dtype=np.int64) * \
                       (self.img_width // self.downsample_factor - 2)
        label_length = np.zeros((batch_len, 1), dtype=np.int64)

        for j, idx in enumerate(curr_batch_idx):
            # 1. Get the image and transpose it
            img = self.data[idx].T
            # 2. Add extra dimenison
            img = np.expand_dims(img, axis=-1)
            # 3. Get the correpsonding label
            text = self.labels[idx]
            # 4. Include the pair only if the captcha is valid
            if is_valid_captcha(text):
                label = [self.char_map[ch] for ch in text]
                batch_images[j] = img
                batch_labels[j] = label
                label_length[j] = len(text)

        batch_inputs = {
            'input_data': batch_images,
            'input_label': batch_labels,
            'input_length': input_length,
            'label_length': label_length,
        }
        return batch_inputs, np.zeros(batch_len).astype(np.float32)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [8]:
# Batch size for training and validation
batch_size = configs.batch_size

# Desired image dimensions
img_width = configs.width
img_height = configs.height

# Factor  by which the image is going to be downsampled
# by the convolutional blocks
downsample_factor = configs.downsample_factor

# Maximum length of any captcha in the data
max_length = configs.max_text_length

# Get a generator object for the training data
train_data_generator = DataGenerator(data=training_data,
                                     labels=training_labels,
                                     char_map=char_to_labels,
                                     batch_size=batch_size,
                                     img_width=img_width,
                                     img_height=img_height,
                                     downsample_factor=downsample_factor,
                                     max_length=max_length,
                                     shuffle=True
                                     )

# Get a generator object for the validation data 
valid_data_generator = DataGenerator(data=validation_data,
                                     labels=validation_labels,
                                     char_map=char_to_labels,
                                     batch_size=batch_size,
                                     img_width=img_width,
                                     img_height=img_height,
                                     downsample_factor=downsample_factor,
                                     max_length=max_length,
                                     shuffle=False
                                     )

In [9]:
class CTCLayer(layers.Layer):
    def __init__(self, name=None, **kwargs):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred, input_length, label_length):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # On test time, just return the computed loss
        return loss


def build_model():
    # Inputs to the model
    input_img = layers.Input(shape=(img_width, img_height, 1),
                             name='input_data',
                             dtype='float32')
    labels = layers.Input(name='input_label', shape=[max_length], dtype='float32')
    input_length = layers.Input(name='input_length', shape=[1], dtype='int64')
    label_length = layers.Input(name='label_length', shape=[1], dtype='int64')

    # First conv block
    x = layers.Conv2D(32,
                      (3, 3),
                      activation='relu',
                      kernel_initializer='he_normal',
                      padding='same',
                      name='Conv1')(input_img)
    x = layers.MaxPooling2D((2, 2), name='pool1')(x)

    # Second conv block
    x = layers.Conv2D(64,
                      (3, 3),
                      activation='relu',
                      kernel_initializer='he_normal',
                      padding='same',
                      name='Conv2')(x)
    x = layers.MaxPooling2D((2, 2), name='pool2')(x)

    # We have used two max pool with pool size and strides of 2.
    # Hence, downsampled feature maps are 4x smaller. The number of
    # filters in the last layer is 64. Reshape accordingly before
    # passing it to RNNs
    new_shape = ((img_width // 4), (img_height // 4) * 64)
    x = layers.Reshape(target_shape=new_shape, name='reshape')(x)
    x = layers.Dense(64, activation='relu', name='dense1')(x)
    x = layers.Dropout(0.2)(x)

    # RNNs
    x = layers.Bidirectional(layers.LSTM(128,
                                         return_sequences=True,
                                         dropout=0.2))(x)
    x = layers.Bidirectional(layers.LSTM(64,
                                         return_sequences=True,
                                         dropout=0.25))(x)

    # Predictions
    x = layers.Dense(len(characters) + 1,
                     activation='softmax',
                     name='dense2',
                     kernel_initializer='he_normal')(x)

    # Calculate CTC
    output = CTCLayer(name='ctc_loss')(labels, x, input_length, label_length)

    if configs.continue_training:
        model = keras.models.load_model(configs.base_model, custom_objects={'CTCLayer': CTCLayer}, compile=False)
        # model.get_layer(name='input_label').shape = (None, configs.max_text_length)
        
    else:
        # Define the model
        model = keras.models.Model(inputs=[input_img,
                                           labels,
                                           input_length,
                                           label_length],
                                   outputs=output,
                                   name=configs.model_name)

    # Optimizer
    sgd = keras.optimizers.SGD(learning_rate=0.002,
                               decay=1e-6,
                               momentum=0.9,
                               nesterov=True,
                               clipnorm=5)

    # Compile the model and return 
    model.compile(optimizer=sgd)
    return model

In [10]:
model = build_model()
model.summary()

Model: "captcha-2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_data (InputLayer)        [(None, 400, 40, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Conv1 (Conv2D)                 (None, 400, 40, 32)  320         ['input_data[0][0]']             
                                                                                                  
 pool1 (MaxPooling2D)           (None, 200, 20, 32)  0           ['Conv1[0][0]']                  
                                                                                                  
 Conv2 (Conv2D)                 (None, 200, 20, 64)  18496       ['pool1[0][0]']          

In [11]:
# Add early stopping
es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                   patience=5,
                                   restore_best_weights=True)

# Train the model
history = model.fit(train_data_generator,
                    validation_data=valid_data_generator,
                    epochs=configs.train_epochs,
                    callbacks=[es])

print("Training completed!")

Epoch 1/40
282/282 [==============================] - 28s 71ms/step - loss: 43.3689 - val_loss: 30.7259
Epoch 2/40
282/282 [==============================] - 19s 66ms/step - loss: 25.3061 - val_loss: 19.5902
Epoch 3/40
282/282 [==============================] - 19s 69ms/step - loss: 19.9116 - val_loss: 17.2756
Epoch 4/40
282/282 [==============================] - 20s 69ms/step - loss: 18.2183 - val_loss: 16.4203
Epoch 5/40
282/282 [==============================] - 21s 74ms/step - loss: 17.3572 - val_loss: 15.8456
Epoch 6/40
282/282 [==============================] - 22s 78ms/step - loss: 16.6977 - val_loss: 15.4923
Epoch 7/40
282/282 [==============================] - 23s 80ms/step - loss: 16.1393 - val_loss: 14.9604
Epoch 8/40
282/282 [==============================] - 23s 81ms/step - loss: 15.6476 - val_loss: 14.8308
Epoch 9/40
282/282 [==============================] - 22s 78ms/step - loss: 15.2947 - val_loss: 14.5704
Epoch 10/40
282/282 [==============================] - 21s 74ms/

KeyboardInterrupt: 

In [ ]:
prediction_model = keras.models.Model(model.get_layer(name='input_data').input,
                                      model.get_layer(name='dense2').output)
prediction_model.summary()

In [ ]:
# A utility to decode the output of the network
def decode_batch_predictions(pred):
    pred = pred[:, :-2]
    input_len = np.ones(pred.shape[0]) * pred.shape[1]

    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred,
                                       input_length=input_len,
                                       greedy=True)[0][0]

    # Iterate over the results and get back the text
    output_text = []
    for res in results.numpy():
        outstr = ''
        for c in res:
            if c < len(characters) and c >= 0:
                outstr += labels_to_char[c]
        output_text.append(outstr)

    # return final text results
    return output_text

In [ ]:
#  Let's check results on some validation samples
for p, (inp_value, _) in enumerate(valid_data_generator):
    bs = inp_value['input_data'].shape[0]
    X_data = inp_value['input_data']
    labels = inp_value['input_label']

    preds = prediction_model.predict(X_data)

    pred_texts = decode_batch_predictions(preds)

    orig_texts = []
    for label in labels:
        text = ''.join([labels_to_char[int(x)] for x in label])
        orig_texts.append(text)

    for i in range(bs):
        print(f'Ground truth: {orig_texts[i]} \t Predicted: {pred_texts[i]}')
    break

# Save the model

In [ ]:
model.save(configs.model_path)